In [18]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Add, MaxPooling1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
DATA_PATH = os.path.join('dataset')
actions = np.array(['one', 'two','three', 'four', 'five', 'six', 'seven', 'eight', 'nine'])
sequence_length = 100
num_features = 126
label_map = {label:num for num, label in enumerate(actions)}

In [4]:
sequences, labels = [], []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    for sequence in np.array(os.listdir(action_path)).astype(int):
        frame_paths = [
            os.path.join(action_path, str(sequence), "{}.npy".format(frame_num))
            for frame_num in range(sequence_length)
        ]
        window = [np.load(frame_path) for frame_path in frame_paths]
        sequences.append(window)
        labels.append(label_map[action])

In [5]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [6]:
log_dir = os.path.join('ResNet_Logs')
callback = TensorBoard(log_dir=log_dir)
ACCURACY_THRESHOLD = 0.95

class MyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, monitor_metric='accuracy', threshold=0.95):
        super(MyCallback, self).__init__()
        self.monitor_metric = monitor_metric
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs={}): 
        current_metric_value = logs.get(self.monitor_metric)
        if current_metric_value is not None and current_metric_value > self.threshold:
            print(f"\nReached {self.threshold * 100:.2f}% {self.monitor_metric}, stopping training!")
            self.model.stop_training = True

tb_callback = TensorBoard(log_dir=log_dir)
my_callback = MyCallback(monitor_metric='accuracy', threshold=ACCURACY_THRESHOLD)

In [34]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Add, MaxPooling1D, GlobalAveragePooling1D, Dense

def residual_block(x, filters, kernel_size=3, stride=1):
    # Shortcut path
    shortcut = x
    
    # Main path
    x = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Resize the shortcut to match the shape of x
    shortcut = Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
    
    # Add shortcut to main path
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    
    return x  # Return the updated tensor


In [35]:
# Define the model
model = Sequential()

# Initial Convolution
model.add(Conv1D(64, kernel_size=7, padding='same', input_shape=(sequence_length, num_features)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Residual Blocks
input_tensor = model.output  # Save the original output tensor

x = residual_block(input_tensor, filters=64)
x = residual_block(x, filters=64)
x = MaxPooling1D(2)(x)

x = residual_block(x, filters=128, stride=2)
x = residual_block(x, filters=128)
x = MaxPooling1D(2)(x)

x = residual_block(x, filters=256, stride=2)
x = residual_block(x, filters=256)
x = MaxPooling1D(2)(x)

x = residual_block(x, filters=512, stride=2)
x = residual_block(x, filters=512)
x = GlobalAveragePooling1D()(x)

# Fully Connected Layers
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
output_tensor = Dense(actions.shape[0], activation='softmax')(x)

# Create a Model with input and output
model = Model(inputs=model.input, outputs=output_tensor)

In [36]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [38]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=8, callbacks=[tb_callback, my_callback, early_stopping])

Epoch 1/100
158/158 [==============================] - 14s 35ms/step - loss: 2.4139 - accuracy: 0.1730 - val_loss: 2.2447 - val_accuracy: 0.1037
Epoch 2/100
158/158 [==============================] - 5s 30ms/step - loss: 2.1785 - accuracy: 0.2079 - val_loss: 2.2201 - val_accuracy: 0.1704
Epoch 3/100
158/158 [==============================] - 5s 29ms/step - loss: 2.0690 - accuracy: 0.2413 - val_loss: 2.0084 - val_accuracy: 0.2907
Epoch 4/100
158/158 [==============================] - 5s 30ms/step - loss: 1.9722 - accuracy: 0.2651 - val_loss: 1.9660 - val_accuracy: 0.2574
Epoch 5/100
158/158 [==============================] - 5s 29ms/step - loss: 1.9431 - accuracy: 0.2706 - val_loss: 2.2679 - val_accuracy: 0.2630
Epoch 6/100
158/158 [==============================] - 5s 29ms/step - loss: 1.8462 - accuracy: 0.3000 - val_loss: 1.8210 - val_accuracy: 0.2944
Epoch 7/100
158/158 [==============================] - 4s 28ms/step - loss: 1.7662 - accuracy: 0.3206 - val_loss: 1.6259 - val_accuracy

In [39]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv1d_28_input (InputLayer)   [(None, 100, 126)]   0           []                               
                                                                                                  
 conv1d_28 (Conv1D)             (None, 100, 64)      56512       ['conv1d_28_input[0][0]']        
                                                                                                  
 batch_normalization_26 (BatchN  (None, 100, 64)     256         ['conv1d_28[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_25 (Activation)     (None, 100, 64)      0           ['batch_normalization_26[0]

In [40]:
model.save('ResNet.h5')

In [41]:
model.load_weights('ResNet.h5')

In [42]:
yhat = model.predict(x_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Train Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

40/40 [==============================] - 1s 9ms/step
Train Accuracy :-> 
91.19047619047619


In [43]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Test Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

17/17 [==============================] - 0s 21ms/step
Test Accuracy :-> 
78.33333333333333
